In [ ]:
# 練習12-6 影像顏色曲線圖*
import cv2
import numpy as np
from matplotlib import pyplot as plt

img_path = r"C:/img/boatGray.bmp"
o = cv2.imread(img_path, cv2.IMREAD_COLOR)
# cv2.imshow("Image", o)
hist = cv2.calcHist([o], [0], None, [256], [0, 255])
plt.plot(hist, color='b', label='Full Image')
print(hist)

In [ ]:
# 練習12-7 遮罩 + 灰階 + 疊加
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt

img_path = r"C:/img/girl.bmp"
img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)

if img is None:
    print(f"找不到圖片：{img_path}")
else:
    # --- 1. 轉灰階 ---
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # --- 2. 建立矩形 mask ---
    h, w = img.shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)

    hh, ww = 200, 200
    mask[200:400,200:400] = 255   # 255 = 被選取的區域

    # --- 3. 計算灰階的直方圖 ---
    full_hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
    mask_hist = cv2.calcHist([gray], [0], mask, [256], [0, 256])

    # --- 4. 將 mask 疊加在原圖上（只留中心灰階其餘為黑） ---
    overlay = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    overlay[mask == 0] = 0  # 非選取區域設為黑色

    # --- 5. 顯示 ---
    fig, axs = plt.subplots(1, 3, figsize=(18, 5))

    axs[0].imshow(cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB))
    axs[0].set_title('Gray Image')
    axs[0].axis('off')


    axs[1].imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    axs[1].set_title('Overlay (mask on image)')
    axs[1].axis('off')

    axs[2].plot(full_hist, color='b', label='Full')
    axs[2].plot(mask_hist, color='r', label='Masked')
    axs[2].set_xlim([0, 256])
    axs[2].set_title('Histogram (Gray)')
    axs[2].legend()

    plt.tight_layout()
    plt.show()


In [ ]:
# 練習12-8 彩色影像顏色曲線圖*
import cv2
import numpy as np
from matplotlib import pyplot as plt

img_path = r"C:/img/girl.bmp"
o = cv2.imread(img_path, cv2.IMREAD_COLOR)
histb = cv2.calcHist([o],[0],None,[256],[0,255])
histg = cv2.calcHist([o],[1],None,[256],[0,255])
histr = cv2.calcHist([o],[2],None,[256],[0,255])
plt.plot(histb, color='b', label='Blue Channel')
plt.plot(histg, color='g', label='Green Channel')
plt.plot(histr, color='r', label='Red Channel')
plt.legend()
plt.show()

In [ ]:
# 練習12-9*
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt

img_path = r"C:/img/girl.bmp"
img = cv2.imread(img_path, cv2.IMREAD_COLOR)

if img is None:
    print(f"找不到圖片：{img_path}")
else:
    h, w = img.shape[:2]

    # --- 1) 建立矩形 mask（依題意沿用固定座標 200:400） ---
    mask = np.zeros((h, w), dtype=np.uint8)
    y0, y1 = 200, min(400, h)
    x0, x1 = 200, min(400, w)
    if y0 < y1 and x0 < x1:
        mask[y0:y1, x0:x1] = 255

    # --- 2) 只留中心彩色，其餘為黑 ---
    result = np.zeros_like(img)
    result[mask == 255] = img[mask == 255]

    # --- 3) B/G/R 三通道直方圖（全圖 vs 遮罩） ---
    colors = ("b", "g", "r")
    full_hists = []
    mask_hists = []
    for ch in range(3):
        full_hists.append(cv2.calcHist([img], [ch], None, [256], [0, 256]))
        mask_hists.append(cv2.calcHist([img], [ch], mask, [256], [0, 256]))

    # --- 4) 視覺化：原圖、遮罩、結果、直方圖 ---
    fig, axs = plt.subplots(1, 4, figsize=(20, 5))

    axs[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axs[0].set_title('Original (Color)')
    axs[0].axis('off')

    axs[1].imshow(mask, cmap='gray')
    axs[1].set_title('Mask (200:400, 200:400)')
    axs[1].axis('off')

    axs[2].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    axs[2].set_title('Result: Center Color, Others Black')
    axs[2].axis('off')

    axs[3].set_title('Histogram B/G/R (Full vs Masked)')
    for ch, c in enumerate(colors):
        axs[3].plot(full_hists[ch], color=c, label=f'{c.upper()} Full')
        axs[3].plot(mask_hists[ch], color=c, label=f'{c.upper()} Mask')
    axs[3].set_xlim([0, 256])
    axs[3].legend()

    plt.tight_layout()
    plt.show()

In [ ]:
# 練習12-10：RGB Color Histogram 的歐式距離比較（Bus_068/099/190）
import os
import cv2
import numpy as np
from itertools import combinations

# 設定影像根目錄與檔名（不含副檔名）
base_dir = r"C:/img"
base_names = ["Bus_068", "Bus_099", "Bus_190"]
try_exts = [".bmp", ".jpg", ".jpeg", ".png"]

# 解析實際存在的檔案路徑
paths = {}
missing = []
for name in base_names:
    found_path = None
    for ext in try_exts:
        p = os.path.join(base_dir, name + ext)
        if os.path.exists(p):
            found_path = p
            break
    if found_path is None:
        missing.append(name)
    else:
        paths[name] = found_path

if missing:
    print("找不到以下檔案（已嘗試 .bmp/.jpg/.jpeg/.png）：", ", ".join(missing))
    print("已找到：", paths)

if len(paths) == 3:
    # 計算每張圖的 RGB 直方圖（BGR in OpenCV；此處統一稱為 RGB）
    # 直方圖 256 bins、[0,256]，並做 L1 正規化以消除尺寸影響 
    feats = {}
    for name, p in paths.items():
        img = cv2.imread(p, cv2.IMREAD_COLOR)
        if img is None:
            print(f"讀取失敗：{p}")
            continue
        # B(0), G(1), R(2)
        hists = []
        for ch in (0, 1, 2):
            hist = cv2.calcHist([img], [ch], None, [256], [0, 256]).astype(np.float32)
            s = hist.sum()
            if s > 0:
                hist /= s
            hists.append(hist.reshape(-1))
        feat = np.concatenate(hists, axis=0)  # 768 維特徵
        feats[name] = feat

    # 建立名稱索引順序
    names = list(feats.keys())
    if len(names) < 3:
        print("可用影像不足 3 張，目前可用：", names)
    else:
        # 距離矩陣
        n = len(names)
        dist_mat = np.zeros((n, n), dtype=np.float32)
        for i in range(n):
            for j in range(n):
                if i == j:
                    dist_mat[i, j] = 0.0
                else:
                    dist_mat[i, j] = np.linalg.norm(feats[names[i]] - feats[names[j]])

        print("影像順序：", names)
        print("\n=== Euclidean Distance Matrix (RGB hist; L1-normalized) ===")
        for i in range(n):
            row = [f"{dist_mat[i, j]:.6f}" for j in range(n)]
            print("[ " + ", ".join(row) + " ]")

        # 兩兩組合排序，找最相似（距離最小）的一對
        pair_dists = []
        for i, j in combinations(range(n), 2):
            pair_dists.append((dist_mat[i, j], names[i], names[j]))
        pair_dists.sort(key=lambda x: x[0])

        print("\n=== Pairwise distances (ascending) ===")
        for d, a, b in pair_dists:
            print(f"{a} vs {b} : {d:.6f}")

        best = pair_dists[0]
        print(f"\n最相似的一對：{best[1]} 與 {best[2]}（距離 {best[0]:.6f}）")
else:
    print("未能取得三張影像，無法進行三圖比較。請確認檔名與路徑。")

In [ ]:
# 練習12-11 Histogram Equalization-Real Case
import cv2
import numpy as np
from matplotlib import pyplot as plt

img_path = r"C:/img/equ.bmp"
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

if img is None:
    raise FileNotFoundError(f"找不到圖片: {img_path}")

# 原圖直方圖
hist_orig = cv2.calcHist([img], [0], None, [256], [0, 256])

# 直方圖均衡化
equ = cv2.equalizeHist(img)

# 均衡化後直方圖
hist_equ = cv2.calcHist([equ], [0], None, [256], [0, 256])

# 顯示
fig, ax = plt.subplots(2, 2, figsize=(12, 8))

# 原圖與均衡化影像
ax[0,0].imshow(img, cmap='gray')
ax[0,0].set_title("Original")
ax[0,0].axis("off")

ax[0,1].imshow(equ, cmap='gray')
ax[0,1].set_title("Equalized")
ax[0,1].axis("off")

# 長條圖直方圖
ax[1,0].bar(np.arange(256), hist_orig.ravel(), color='blue')
ax[1,0].set_title("Original Histogram")
ax[1,0].set_xlim([0, 256])
ax[1,0].set_ylim([0, 8000])  # 固定 y 軸
ax[1,0].grid(alpha=0.3)

ax[1,1].bar(np.arange(256), hist_equ.ravel(), color='green')
ax[1,1].set_title("Equalized Histogram")
ax[1,1].set_xlim([0, 256])
ax[1,1].set_ylim([0, 8000])  # 固定 y 軸
ax[1,1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# 額外統計
print(f"Original Mean={img.mean():.2f}, Std={img.std():.2f}")
print(f"Equalized Mean={equ.mean():.2f}, Std={equ.std():.2f}")


In [ ]:
# 練習12-13 imshow()-1*
import cv2
from matplotlib import pyplot as plt

# 讀取影像
img_path = r"C:/img/girl.bmp"
img = cv2.imread(img_path, cv2.IMREAD_COLOR)  # 讀 BGR 彩色圖

if img is None:
    raise FileNotFoundError(f"找不到圖片: {img_path}")

# BGR 轉 RGB
imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# 顯示左右對照圖
plt.figure("顯示結果", figsize=(10,5))

plt.subplot(1,2,1)
plt.imshow(img)  # 原 BGR 影像 (用 matplotlib 會顏色偏藍)
plt.axis('off')
plt.title("Original BGR")

plt.subplot(1,2,2)
plt.imshow(imgRGB)  # 轉成 RGB 後顏色正常
plt.axis('off')
plt.title("Converted RGB")

plt.tight_layout()
plt.show()


In [ ]:
# 練習12-14 imshow()-2
import cv2
from matplotlib import pyplot as plt

# 讀取彩色圖
img_path = r"C:/img/girl.bmp"
bgr = cv2.imread(img_path)  # 彩色BGR
if bgr is None:
    raise FileNotFoundError(f"找不到圖片: {img_path}")

# 彩色轉 RGB
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

# 彩色轉灰階
gray_from_color = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)

# 假設原圖已經是灰階（如果原本就是灰階）
gray = gray_from_color.copy()  # 這裡就用同一張灰階影像

# 顯示四種影像
plt.figure("四種顯示", figsize=(12,8))

plt.subplot(2,2,1)
plt.imshow(bgr)  # BGR直接用plt顯示，顏色錯誤
plt.axis('off')

plt.subplot(2,2,2)
plt.imshow(bgr,cmap=plt.cm.gray ),plt.axis('off') #彩色BGR轉灰階錯誤，參數cmap=plt.cm.gray
plt.axis('off')

plt.subplot(2,2,3)
plt.imshow(gray),plt.axis('off') #灰階用RGB模式顯示錯誤
plt.axis('off')

plt.subplot(2,2,4)
plt.imshow(gray, cmap=plt.cm.gray)  # 原本灰階影像
plt.axis('off')

plt.tight_layout()
plt.show()
